In [ ]:
####### OEA configuration #############
oea_storage_account = 'yourstorageaccount'
oea_keyvault = 'yourkeyvault'
oea_timezone = 'US/Eastern'
#######################################

from delta.tables import DeltaTable
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from pyspark.sql.dataframe import DataFrame
import logging
import pandas as pd
import sys
import re
import json
import datetime
import pytz
import random
import io
import requests

logger = logging.getLogger('OEA')

class OEA:
    """ OEA (Open Education Analytics) framework simplifies the process of working with large data sets within the context of a lakehouse architecture.
        Definition of terms used throughout this codebase:
        path - a complete or partial folder or file path (does not include details like scheme or domain name as found in a URL). Ex: contosos/v0.1/students
        entity_path - a path that ends with a folder that contains entity data. Ex: contoso/v0.1/students
        dataset_path - a path that ends with a folder that contains entity folders (entity parent folder). Ex: contoso/v0.1
        url - includes scheme and domain name. Ex: abfss://stage1@storageaccount.dfs.core.windows.net/contoso/v0.1/students

    """
    DELTA_BATCH_DATA = 'delta_batch_data'
    ADDITIVE_BATCH_DATA = 'additive_batch_data'
    SNAPSHOT_BATCH_DATA = 'snapshot_batch_data'

    def __init__(self, workspace='dev', logging_level=logging.INFO, storage_account=None, keyvault=None, timezone=None):
        self.keyvault_linked_service = 'LS_KeyVault'
        self.salt_secret_name = 'oeaSalt'
        self.salt = None
        self.workspace = workspace
        self.storage_account = oea_storage_account
        self.keyvault = oea_keyvault
        self.timezone = oea_timezone

        # pull in override values if any were passed in
        if workspace: self.workspace = workspace
        if storage_account: self.storage_account = storage_account
        if keyvault: self.keyvault = keyvault 
        if timezone: self.timezone = timezone
        if logging_level: self.logging_level = logging_level    

        self._initialize_logger(logging_level)
        self.set_workspace(self.workspace)
        spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true") # more info here: https://learn.microsoft.com/en-us/azure/synapse-analytics/spark/optimize-write-for-apache-spark
        logger.info("OEA initialized.")

    def _initialize_logger(self, logging_level):
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        for handler in logging.getLogger().handlers:
            handler.setFormatter(formatter)           
        # Customize log level for all loggers
        logging.getLogger().setLevel(logging_level)        

    def _get_secret(self, secret_name):
        """ Retrieves the specified secret from the keyvault.
            This method assumes that the keyvault linked service has been setup and is accessible.
        """
        sc = SparkSession.builder.getOrCreate()
        token_library = sc._jvm.com.microsoft.azure.synapse.tokenlibrary.TokenLibrary
        value = token_library.getSecret(self.keyvault, secret_name, self.keyvault_linked_service)        
        return value

    def _get_salt(self):
        if not self.salt:
            self.salt = self._get_secret(self.salt_secret_name)
        return self.salt

    def set_workspace(self, workspace_name):
        """ Allows you to use OEA against your workspace
            (eg, you specify Jon as workspace_name, then instead of reading in from stage1 OEA will use workspace/Jon/stage1
        """
        
        if workspace_name == 'prod' or workspace_name == 'production':
            self.workspace = 'prod'
            self.stage1 = 'abfss://stage1@' + self.storage_account + '.dfs.core.windows.net'
            self.stage2 = 'abfss://stage2@' + self.storage_account + '.dfs.core.windows.net'
            self.stage3 = 'abfss://stage3@' + self.storage_account + '.dfs.core.windows.net'
        elif workspace_name == 'dev' or workspace_name == 'development':
            self.workspace = 'dev'
            self.stage1 = f'abfss://oea@{self.storage_account}.dfs.core.windows.net/dev/stage1'
            self.stage2 = f'abfss://oea@{self.storage_account}.dfs.core.windows.net/dev/stage2'
            self.stage3 = f'abfss://oea@{self.storage_account}.dfs.core.windows.net/dev/stage3'
        else:
            self.workspace = workspace_name
            self.stage1 = f'abfss://oea@{self.storage_account}.dfs.core.windows.net/sandboxes/{workspace_name}/stage1'
            self.stage2 = f'abfss://oea@{self.storage_account}.dfs.core.windows.net/sandboxes/{workspace_name}/stage2'
            self.stage3 = f'abfss://oea@{self.storage_account}.dfs.core.windows.net/sandboxes/{workspace_name}/stage3'
        logger.info(f'Now using workspace: {self.workspace}')

    def to_url(self, path):
        """ Converts the given path into a valid url.
            eg, convert_path('stage1/contoso_sis/student') # returns abfss://stage1@storageaccount.dfs.core.windows.net/contoso_sis/student
            [Note that the url returned will include the sandbox location if a workspace has been set; for example, abfss://oea@storageaccount.dfs.core.windows.net/sandboxes/sam/stage1/contoso_sis/student]
        """
        if not path or path == '': raise ValueError('Specified path cannot be empty.')
        if path.startswith('abfss://'): return path # if a url is given, just return that same url (allows to_url to be invoked just in case translation may be needed)
        path_args = path.split('/')
        stage = path_args.pop(0)
        if stage == 'stage1': stage = self.stage1
        elif stage == 'stage2': stage = self.stage2
        elif stage == 'stage3': stage = self.stage3
        else: raise ValueError("Path must begin with either 'stage1', 'stage2', or 'stage3'")
        url = f"{stage}/{'/'.join(path_args)}"
        logger.debug(f'to_url: {url}')
        return url      

    def parse_path(self, path):
        """ Parses a path that looks like one of the following:
                ms_insights/v0.1
                ms_insights/v0.1/students

                stage1/Transactional/ms_insights/v0.1
                stage1/Transactional/ms_insights/v0.1/students
            (the path must either be the path to a specific entity, or the path to the parent folder containing entities)
            and returns a dictionary like one of the following:
                {'stage': 'stage1', 'stage_num': '1', 'category': 'Transactional', 'source_system': 'contoso_sis', 'entity': None, 'entity_list': ['studentattendance'], 'entity_path': None, 'entity_parent_path': 'stage1/Transactional/contoso_sis/v0.1'}
                {'stage': 'stage1', 'stage_num': '1', 'category': 'Transactional', 'source_system': 'contoso_sis', 'entity': 'studentattendance', 'entity_list': None, 'entity_path': 'stage1/Transactional/contoso_sis/v0.1/studentattendance', 'entity_parent_path': 'stage1/Transactional/contoso_sis/v0.1'}

            This method assumes the standard OEA data lake, in which paths have this structure: <stage number>/<category>/<source system>/<optional version and partitioning>/<entity>/<either batch_data folder or _delta_log>
        """
        if type(path) is dict: return path # this means the path was already parsed
        
        ar = path.split('/')
        path_dict = {'stage':ar[0], 'stage_num':ar[0][-1], 'category':ar[1], 'source_system':ar[2], 'entity':None, 'entity_list':None, 'entity_path':None, 'entity_parent_path':None}

        folders = self.get_folders(self.to_url(path))

        # Identify an entity folder by the presence of the "_delta_log" folder in stage2 and stage3
        if (path_dict['stage_num'] == '1' and ('additive_batch_data' in folders or 'delta_batch_data' in folders or 'snapshot_batch_data' in folders)) or ((path_dict['stage_num'] == '2' or path_dict['stage_num'] == '3') and '_delta_log' in folders):
            path_dict['entity'] = ar[-1]
            path_dict['entity_path'] = path
            path_dict['entity_parent_path'] = '/'.join(ar[0:-1]) # eg, stage1/Transactional/contoso_sis/v0.1
        else:
            path_dict['entity_list'] = folders
            path_dict['entity_parent_path'] = path

        if path_dict['stage'] == 'stage2':
            abbrev = path_dict['category'][0].lower() # either 'i' for Ingested or 'r' for Refined
            path_dict['sdb_name'] = f'sdb_{self.workspace}_s{path_dict["stage_num"]}{abbrev}_{path_dict["source_system"].lower()}' # name of the sql db for this source (use lower case to match the naming for lake db)
            path_dict['ldb_name'] = f'ldb_{self.workspace}_s{path_dict["stage_num"]}{abbrev}_{path_dict["source_system"].lower().replace("-","_")}' # name of the lake db for this source (spark will automatically lower case the name of the db, but we're doing it here to be explicit)
        else:
            path_dict['sdb_name'] = f'sdb_{self.workspace}_s{path_dict["stage_num"]}_{path_dict["source_system"].lower()}' # name of the sql db for this source (use lower case to match the naming for lake db)
            path_dict['ldb_name'] = f'ldb_{self.workspace}_s{path_dict["stage_num"]}_{path_dict["source_system"].lower().replace("-","_")}' # name of the lake db for this source (spark will automatically lower case the name of the db, but we're doing it here to be explicit)
        
        path_dict['between_path'] = '/'.join(path_dict['entity_parent_path'].split('/')[2:]) # strip off the first 2 args in the entity parent path (eg, strip off stage1/Transactional which leaves contoso_sis/v0.1)

        m = re.match(r'.*\/(v[^\/]+).*', path_dict['between_path'])
        if m:
            path_dict['version'] = m.group(1)
            # Append the version number to the db names. First replace the '.' char with a 'p' if necessary (because a '.' is not allowed in the db name)
            safe_version = re.sub('\.', 'p', path_dict["version"])
            path_dict['sdb_name'] = f'{path_dict["sdb_name"]}_{safe_version}'
            path_dict['ldb_name'] = f'{path_dict["ldb_name"]}_{safe_version}'
        else:
            path_dict['version'] = None

        return path_dict
    
    def rm_if_exists(self, path, recursive_remove=True):
        """ Remove a folder if it exists (defaults to use of recursive removal). """
        try:
            mssparkutils.fs.rm(self.to_url(path), recursive_remove)
        except Exception as e:
            pass

    def delete(self, path):
        """ Delete a folder and everything in it. 
            This only works for non-production workspaces - to prevent accidental deletion of production data.
        """
        if self.workspace == 'prod':
            raise ValueError("Your current workspace is 'prod'. The 'delete' method cannot be used against the production workspace.") 
        else:
            self.rm_if_exists(path, True)

    def ls(self, path):
        """ List the contents of the given path. """
        url = self.to_url(path)
        folders = []
        files = []
        try:
            items = mssparkutils.fs.ls(url)
            for item in items:
                if item.isFile:
                    files.append(item.name)
                elif item.isDir:
                    folders.append(item.name)
        except Exception as e:
            logger.warning("[OEA] Could not peform ls on specified path: " + path + "\nThis may be because the path does not exist.")
        return (folders, files)

    def path_exists(self, path):
        """ Returns true if path exists, false if it doesn't (no exception will be thrown). 
            eg, path_exists('stage1/mytest/v1.0')
        """
        try:
            items = mssparkutils.fs.ls(self.to_url(path))
        except Exception as e:
            # This Exception comes as a generic Py4JJavaError that occurs when the path specified is not found.
            return False
        return True

    def get_stage_num(self, path):
        m = re.match(r'.*stage(\d)/.*', path)
        if m:
            return m.group(1)
        else:
            raise ValueError("Path must begin with either 'stage1', 'stage2', or 'stage3'")

    def get_folders(self, path):
        """ Return the list of folders found in the given path. """
        dirs = []
        try:
            items = mssparkutils.fs.ls(self.to_url(path))
            for item in items:
                #print(item.name, item.isDir, item.isFile, item.path, item.size)
                if item.isDir:
                    dirs.append(item.name)
        except Exception as e:
            logger.warning("[OEA] Could not get list of folders in specified path: " + path + "\nThis may be because the path does not exist.")
        return dirs

    def get_folder_size(self, path):
        """ Return the list of folders found in the given path. """
        result = 0
        try:
            items = mssparkutils.fs.ls(self.to_url(path))
            for item in items:
                result += item.size
        except Exception as e:
            logger.warning("[OEA] Could not get list of folders in specified path: " + path + "\nThis may be because the path does not exist.")
        return result

    def contains_children(self, path):
        """Returns a boolean if the folder has children"""
        try:
            items = mssparkutils.fs.ls(self.to_url(path))
            if len(items)>0:
                return True
        except Exception as e:
            #If an exception is thrown the parent folder doesn't exist
            #return False
            logger.warning("[OEA] Could not get list of folders in specified path: " + path + "\nThis may be because the path does not exist.")

        return False

    def get_latest_runtime(self, path, pattern):
        """Gets the datetime of the latest runtime"""
        dirs = self.get_folders(path)
        if len(dirs) == 0:
            return None

        latest_runtime = datetime.datetime.strptime(dirs[0], pattern)
        for i in range(len(dirs)-1):
            curr_runtime = datetime.datetime.strptime(dirs[i+1], pattern)
            if(curr_runtime > latest_runtime):
                latest_runtime = curr_runtime
        return latest_runtime

    def get_latest_folder(self, path):
        """ Gets the last folder listed in the given path. """
        folders = self.get_folders(path)
        if len(folders) > 0: return folders[-1]
        else: return None

    def contains_batch_folder(self, path):
        for name in self.get_folders(self.to_url(path)):
            if name == 'additive_batch_data' or name == 'snapshot_batch_data' or name == 'delta_batch_data':
                return True
        return False

    def get_batch_info(self, source_path):
        """ Given a source data path, returns a tuple with the batch type (based on the name of the folder) and file type (based on a file extension) 
            eg, get_batch_info('stage1/Transactional/sis/v1.0/students') # returns ('snapshot', 'csv')
        """
        url = self.to_url(source_path)
        folders = self.get_folders(url) #expects to find one of: additivie_batch_data, snapshot_batch_data, delta_batch_data, delete_batch_data
        batches = []
        for folder in folders:
            batch_type = folder.split('_')[0]
            if batch_type == 'additive' or batch_type == 'snapshot' or batch_type == 'delta' or batch_type == 'delete':
                rundate_dir = self.get_latest_folder(f'{url}/{folder}')
                data_files = self.ls(f'{url}/{folder}/{rundate_dir}')[1]
                file_extension = data_files[0].split('.')[1]
                batches.append((batch_type, file_extension))
        
        #if delete is one of the batches, reorganize the folders so it is the last batch
        #This is done because on the initial load at least one batch has to be present before
        #Deletion will work properly
        if(len(batches) > 1):
            delete_index = -1
            for i in range(len(batches)):
                if batches[i][0] == "delete":
                    delete_index = i
            if delete_index > -1:
                batches = [*batches[0: delete_index],*batches[delete_index+1:len(batches)+1], batches[ delete_index]]
        return batches       

    def load(self, path):
        df = spark.read.format('delta').load(self.to_url(path))
        return df        

    def display(self, path, limit=4):
        df = spark.read.format('delta').load(self.to_url(path))
        display(df.limit(limit))
        return df

    def show(self, path, limit=4):
        df = spark.read.format('delta').load(self.to_url(path))
        df.show(limit)
        return df

    def fix_column_names(self, df):
        """ Fix column names to satisfy the Parquet naming requirements by substituting invalid characters with an underscore. """
        df_with_valid_column_names = df.select([F.col(col).alias(self.fix_column_name(col)) for col in df.columns])
        return df_with_valid_column_names

    def fix_column_name(self, column_name):
        return re.sub("[ ,;{}()\n\t=]+", "_", column_name) 

    def to_spark_schema(self, schema):#: list[list[str]]):
        """ Creates a spark schema from a schema specified in the OEA schema format. 
            Example:
            schemas['Person'] = [['Id','string','hash'],
                                    ['CreateDate','timestamp','no-op'],
                                    ['LastModifiedDate','timestamp','no-op']]
            to_spark_schema(schemas['Person'])
        """
        fields = []
        for col_schema in schema:
            col_name = col_schema[0]
            col_dtype = col_schema[1]
            fields.append(StructField(col_name, globals()[col_dtype.lower().capitalize() + "Type"](), True))
        spark_schema = StructType(fields)
        return spark_schema

    def get_text_from_path(self, path):
        txt = mssparkutils.fs.head(oea.to_url(path), 9000000)
        return txt

    def get_metadata_from_url(self, url):
        csv_str = requests.get(url).text
        metadata = self.parse_metadata_from_csv(csv_str)
        return metadata   

    def get_metadata_from_path(self, path):
        csv_str = self.get_text_from_path(path + '/metadata.csv')
        metadata = self.parse_metadata_from_csv(csv_str)
        return metadata                   

    def land_metadata_from_url(self, metadata_url, dataset_path):
        metadata_str = requests.get(metadata_url).text
        self.write(metadata_str, self._metadata_path(dataset_path))

    def _metadata_path(self, dataset_path):
        return f'stage2/Ingested/{dataset_path}/metadata.csv'

    def parse_metadata_from_csv(self, csv_str):
        """ Parses out metadata from a csv string and returns the metadata dictionary. """
        metadata = {}
        current_entity = ''
        header = None
        for line in csv_str.splitlines():
            line = line.strip()
            # skip empty lines, lines that start with # (because these are comments), and lines with only commas (which is what happens if someone uses excel and leaves a row blank) 
            if len(line) == 0 or line.startswith('#') or re.match(r'^,+$', line): continue
            ar = line.split(',')

            if not header:
                header = []
                for column_name in ar:
                    header.append(re.sub("[ ,;{}()\n\t=]+", "_", column_name))
                continue
            
            # check for the start of a new entity definition
            if ar[0] != '':
                current_entity = ar[0]
                metadata[current_entity] = []
            # an attribute row must have an attribute name in the second column
            elif len(ar[1]) > 0:
                ar = ar[1:] # remove the first element because it will be blank
                ar[0] = self.fix_column_name(ar[0]) # remove spaces and other illegal chars from column names
                metadata[current_entity].append(ar)
            else:
                logger.info('Invalid metadata row: ' + line)
        return metadata

    def write(self, data_str, destination_path_and_filename):
        """ Writes the given data string to a file on blob storage """
        destination_url = self.to_url(destination_path_and_filename)
        mssparkutils.fs.put(destination_url, data_str, True) # Set the last parameter as True to create the file if it does not exist    

    def create_run_date(self):
        rundate = datetime.datetime.now().replace(microsecond=0) # use UTC for the datetime because when parsing it out later, spark's to_timestamp() assumes the local machine's timezone, and the timezone for the spark cluster will be UTC
        return rundate

    def land(self, data, entity_path, filename=None, batch_data_type=DELTA_BATCH_DATA, rundate=None):
        """ Lands data in the given entity_path, adding a rundate folder.
            eg, land(data, 'contoso/v0.1/students', 'students.csv', oea.SNAPSHOT_BATCH_DATA)
            The data must be either a string, a pyspark DataFrame, or a pandas DataFrame.
        """
        if not rundate:
            rundate = self.create_run_date()
            # This is a workaround for a defect in spark that occurs when using structured streaming with json files. For some reason the readStream method does not allow colons when reading from json source.
            # (The exception it throws is --> IllegalArgumentException: java.net.URISyntaxException: Relative path in absolute URI: rundate=2022-11-26 21:49:05)
            file_extension = filename.split('.')[1]
            if file_extension == 'json':
                rundate = str(rundate).replace(':', '-')

        sink_path = f'stage1/Transactional/{entity_path}/{batch_data_type}/rundate={rundate}'
        if isinstance(data, DataFrame):
            data.write.mode('overwrite').format('parquet').save(self.to_url(sink_path))
        elif isinstance(data, pd.DataFrame):
            df = spark.createDataFrame(data)
            df.write.mode('overwrite').format('parquet').save(self.to_url(sink_path))            
        elif isinstance(data, str):
            self.write(data, f'{sink_path}/{filename}')
        else:
            raise ValueError(f'The given data is in an unsupported format.') 
        return sink_path                  

    def upsert(self, df, destination_path, primary_key='id'):
        """ Upserts the data in the given dataframe into the specified destination using the given primary_key_column to identify the updates.
            If there is no delta table found in the destination_path, one will be created.    
        """
        destination_url = self.to_url(destination_path)
        df = self.fix_column_names(df)
        df = df.dropDuplicates([primary_key])
        if DeltaTable.isDeltaTable(spark, destination_url):
            delta_table_sink = DeltaTable.forPath(spark, destination_url)
            #delta_table_sink.alias('sink').option('mergeSchema', 'true').merge(df.alias('updates'), f'sink.{primary_key} = updates.{primary_key}').whenMatchedUpdateAll().whenNotMatchedInsertAll()
            delta_table_sink.alias('sink').merge(df.alias('updates'), f'sink.{primary_key} = updates.{primary_key}').whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
        else:
            logger.debug('No existing delta table found. Creating delta table.')
            df.write.format('delta').save(destination_url)

    def overwrite(self, df, destination_path, primary_key='id'):
        """ Overwrites the existing delta table with the given dataframe.
            If there is no delta table found in the destination_path, one will be created.    
        """
        destination_url = self.to_url(destination_path)
        df = self.fix_column_names(df)
        df = df.dropDuplicates([primary_key])
        df.write.format('delta').mode('overwrite').save(destination_url)  # https://docs.delta.io/latest/delta-batch.html#overwrite        

    def append(self, df, destination_path, primary_key='id'):
        """ Appends the given dataframe to the delta table in the specified destination.
            If there is no delta table found in the destination_path, one will be created.    
        """
        destination_url = self.to_url(destination_path)
        df = self.fix_column_names(df)
        df = df.dropDuplicates([primary_key])
        if DeltaTable.isDeltaTable(spark, destination_url):
            df.write.format('delta').mode('append').save(destination_url)  # https://docs.delta.io/latest/delta-batch.html#append
        else:
            logger.debug('No existing delta table found. Creating delta table.')
            df.write.format('delta').save(destination_url)
    
    def delete_rows(self, df, destination_path, primary_key='id', batch_id=''):
        """Removes the entities in the given dataframe from the delta table in the specified destination
            Will drop and refresh the lake database table
        """
        ref = f'{destination_path.split("/")[-1]}{batch_id}'
        print(ref)
        df.createOrReplaceGlobalTempView(ref)
        source_dict = self.parse_path(destination_path)
        db_name = source_dict['ldb_name']
        entity_name = source_dict['entity']
        
        url = self.to_url(destination_path)
        dt = spark.read.format("delta").load(url)
        dt_columns = ', '.join(f'{db_name}.{entity_name}.{col}'for col in dt.columns)

        #Synapse does not yet support subqueries inside of a delete statement, so we found the quickest
        #method for deleting entities is joining the dataframe with the table and filtering out the entities
        #that exist in the dataframe (keeping only the entities that exist in the table)
        df1 = spark.sql(f"Select {dt_columns} FROM {db_name}.{entity_name} left outer join global_temp.{ref} on global_temp.{ref}.{primary_key} = {db_name}.{entity_name}.{primary_key} where isnull(global_temp.{ref}.{primary_key})")
        df1.write.format('delta').mode('overwrite').save(url)

        #Refresh the table, otherwise the changes don't seem to be visible
        spark.sql(f"DROP TABLE IF EXISTS {db_name}.{entity_name}")
        spark.sql(f"create table if not exists {db_name}.{entity_name} using DELTA location '{self.to_url(source_dict['entity_path'])}'")

    def process(self, source_path, foreach_batch_function, options={}):
        """ This simplifies the process of using structured streaming when processing transformations.
            Provide a source_path and a function that receives a dataframe to work with (which will be a dataframe with data from the given source_path).
            Use it like this...
            def refine_contoso_dataset(df_source):
                metadata = oea.get_metadata_from_url('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/gene/v0.7dev/modules/module_catalog/Student_and_School_Data_Systems/metadata.csv')
                df_pseudo, df_lookup = oea.pseudonymize(df, metadata['studentattendance'])
                oea.upsert(df_pseudo, 'stage2/Refined/contoso_sis/v0.1/studentattendance/general')
                oea.upsert(df_lookup, 'stage2/Refined/contoso_sis/v0.1/studentattendance/sensitive')
            oea.process('stage2/Ingested/contoso_sis/v0.1/studentattendance', refine_contoso_dataset)             
        """
        if not self.path_exists(source_path):
            raise ValueError(f'The given path does not exist: {source_path} (which resolves to: {self.to_url(source_path)})') 

        def wrapped_function(df, batch_id):
            df.persist() # cache the df so it doesn't get read in multiple times when we write to multiple destinations. See: https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#foreachbatch
            foreach_batch_function(df, batch_id)
            df.unpersist()

        spark.sql("set spark.sql.streaming.schemaInference=true")
        #source_path = source_path.replace(':', '\:')
        print(f"source_path is: {source_path}")
        streaming_df = spark.readStream.load(self.to_url(source_path), **options)
        # for more info on append vs complete vs update modes for structured streaming: https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#basic-concepts
        query = streaming_df.writeStream.format('delta').outputMode('append').trigger(once=True).option('checkpointLocation', self.to_url(source_path) + '/_checkpoints').foreachBatch(wrapped_function).start()
        query.awaitTermination()   # block until query is terminated, with stop() or with error; A StreamingQueryException will be thrown if an exception occurs.
        number_of_new_inbound_rows = query.lastProgress["numInputRows"]
        logger.info(f'Number of new inbound rows processed: {number_of_new_inbound_rows}')
        logger.debug(query.lastProgress)
        return number_of_new_inbound_rows

    def ingest_all(self, dataset_path, primary_key='id', options={}):
        """ Ingests all the entities in the given source_path.
            CSV files are expected to have a header row by default, and JSON files are expected to have complete JSON docs on each row in the file.
            To specify options that are different from these defaults, use the options param.
            eg, ingest('contoso_sis/v0.1') # ingests all entities found in that path
            eg, ingest('contoso_sis/v0.1', options={'header':False}) # for CSV files that don't have a header        
        """
        folders = self.get_folders(self.to_url(f'stage1/Transactional/{dataset_path}'))
        number_of_new_inbound_rows = 0
        for entity_name in folders:
            number_of_new_inbound_rows += self.ingest(f'{dataset_path}/{entity_name}', primary_key, options)
        return number_of_new_inbound_rows

    def ingest(self, entity_path, primary_key='id', options={}):
        """ Ingests the data for the entity in the given path.
            CSV files are expected to have a header row by default, and JSON files are expected to have complete JSON docs on each row in the file.
            To specify options that are different from these defaults, use the options param.
            eg, ingest('contoso_sis/v0.1/students') # ingests all entities found in that path
            eg, ingest('contoso_sis/v0.1/students', options={'header':False}) # for CSV files that don't have a header
        """
        primary_key = self.fix_column_name(primary_key) # fix the column name, in case it has a space in it or some other invalid character
        ingested_path = f'stage2/Ingested/{entity_path}'
        raw_path = f'stage1/Transactional/{entity_path}'

        if not self.path_exists(raw_path):
            logger.error(f'Failed to ingest data because the given source data was not found where expected: {raw_path}')
            return

        batches = self.get_batch_info(raw_path)
        number_of_inbound_changes = 0
        for batch in batches:
            batch_type = batch[0]
            source_data_format = batch[1]
            logger.info(f'Ingesting from: {raw_path}, batch type of: {batch_type}, source data format of: {source_data_format}')
            source_url = self.to_url(f'{raw_path}/{batch_type}_batch_data')

            if oea.get_folder_size(f'{source_url}/{self.get_latest_folder(source_url)}') > 0:
                if batch_type == 'snapshot'or batch_type=='additive': source_url = f'{source_url}/{self.get_latest_folder(source_url)}' 
                    
                logger.debug(f'Processing {batch_type} data from: {source_url} and writing out to: {ingested_path}')
                if batch_type == 'snapshot':
                    def batch_func(df, batch_id): self.overwrite(df, ingested_path, primary_key)
                elif batch_type == 'additive':
                    def batch_func(df, batch_id): self.append(df, ingested_path, primary_key)
                elif batch_type == 'delta':
                    def batch_func(df, batch_id): self.upsert(df, ingested_path, primary_key)
                elif batch_type == "delete":
                    def batch_func(df, batch_id): self.delete_rows(df, ingested_path, primary_key, batch_id)
                else:
                    raise ValueError("No valid batch folder was found at that path (expected to find a single folder with one of the following names: snapshot_batch_data, additive_batch_data, or delta_batch_data). Are you sure you have the right path?")                      

                if options == None: options = {}
                options['format'] = source_data_format # eg, 'csv', 'json'
                if source_data_format == 'csv' and (not 'header' in options or options['header'] == None): options['header'] = True  # default to expecting a header in csv files
                if source_data_format == 'json' and (not 'multiline' in options or options['multiline'] == None): options['multiline'] = True # default to expecting multiline formatted json data

                number_of_new_inbound_rows = self.process(source_url, batch_func, options)
                if number_of_new_inbound_rows > 0:    
                    self.add_to_lake_db(ingested_path)
                number_of_inbound_changes += number_of_new_inbound_rows
        return number_of_inbound_changes

    def query(self, source_path, query_str, criteria_str=None):
        df = self.load(source_path)
        sqlContext.registerDataFrameAsTable(df, 'tmp_source_table')
        if criteria_str:
            query = f'{query_str} from tmp_source_table where {criteria_str}'
        else:
            query = f'{query_str} from tmp_source_table'
        df = sqlContext.sql(query)
        return df       

    def get_latest_changes(self, source_path, sink_path):
        """ Returns a dataframe representing the changes in the source data based on the max rundate in the sink data. 
            If the sink path is not found, all of the data from the source_path is returned (the assumption is that the sink delta table is being created for the first time).
            eg, get_latest_changes('stage2/Ingested/contoso/v0.1/students', 'stage2/Refined/contoso/v0.1/students')
        """   
        maxdatetime = None
        try:
            sink_df = self.query(sink_path, 'select max(rundate) maxdatetime')
            maxdatetime = sink_df.first()['maxdatetime']
        except AnalysisException as e:
            # This means that there is no delta table at the sink_path yet.
            # We'll assume that the sink delta table is being created for the first time, meaning that all of the source data should be returned.
            pass

        changes_df = self.load(source_path)
        if maxdatetime:
            # filter the source table for the latest changes (using the max rundate in the destination table as the watermark)
            changes_df = changes_df.where(f"rundate > '{maxdatetime}'")        
        return changes_df

    def refine_all(self, dataset_path, metadata=None, primary_key='id'):
        """ Refines all the entities in the given dataset_path.
            CSV files are expected to have a header row by default, and JSON files are expected to have complete JSON docs on each row in the file.
            To specify options that are different from these defaults, use the options param.
            eg, ingest('contoso_sis/v0.1') # ingests all entities found in that path
            eg, ingest('contoso_sis/v0.1', options={'header':False}) # for CSV files that don't have a header        
        """
        folders = self.get_folders(self.to_url(f'stage2/Ingested/{dataset_path}'))
        number_of_new_inbound_rows = 0
        for entity_name in folders:
            number_of_new_inbound_rows += self.refine(f'{dataset_path}/{entity_name}', metadata, primary_key)
        return number_of_new_inbound_rows

    def refine(self, entity_path, metadata=None, primary_key='id'):
        source_path = f'stage2/Ingested/{entity_path}'
        primary_key = self.fix_column_name(primary_key) # fix the column name, in case it has a space in it or some other invalid character
        path_dict = self.parse_path(source_path)
        sink_general_path = path_dict['entity_parent_path'].replace('Ingested', 'Refined') + '/general/' + path_dict['entity']
        sink_sensitive_path = path_dict['entity_parent_path'].replace('Ingested', 'Refined') + '/sensitive/' + path_dict['entity'] + '_lookup'
        if not metadata:
            all_metadata = self.get_metadata_from_path(path_dict['entity_parent_path'])
            metadata = all_metadata[path_dict['entity']]

        df_changes = self.get_latest_changes(source_path, sink_general_path)
        spark_schema = self.to_spark_schema(metadata)
        df_changes = self.modify_schema(df_changes, spark_schema)        

        if df_changes.count() > 0:
            df_pseudo, df_lookup = self.pseudonymize(df_changes, metadata)
            self.upsert(df_pseudo, sink_general_path, f'{primary_key}_pseudonym') # todo: remove this assumption that the primary key will always be hashed during pseduonymization
            self.upsert(df_lookup, sink_sensitive_path, primary_key)    
            self.add_to_lake_db(sink_general_path)
            self.add_to_lake_db(sink_sensitive_path)
            logger.info(f'Processed {df_changes.count()} updated rows from {source_path} into stage2/Refined')
        else:
            logger.info(f'No updated rows in {source_path} to process.')
        return df_changes.count()

    def modify_schema(self, df, target_spark_schema):
        """ Transforms the given dataframe by adding appropriate schema to all the columns according to the given target schema
            This method only handles primitive datatypes in this version.
        """
        for col_name in df.columns:
            if col_name in target_spark_schema.names:
                target_dtype = target_spark_schema[col_name].dataType
                df = df.withColumn(col_name, F.col(col_name).cast(target_dtype))
        return df

    def delete_dataset(self, dataset_path):
        oea.delete(f'stage1/Transactional/{dataset_path}')
        oea.delete(f'stage2/Ingested/{dataset_path}')
        oea.delete(f'stage2/Refined/general/{dataset_path}')
        oea.delete(f'stage2/Refined/sensitive/{dataset_path}_lookup') # need to add the _lookup to the end of the dataset name
        # todo: implement the logic to do this...
        #oea.drop_lake_db('ldb_sam_s2i_contoso_v0p1')
        #oea.drop_lake_db('ldb_sam_s2r_contoso_v0p1')        

    def load_csv(self, source_path, header=True):
        """ Loads a csv file as a dataframe based on the path specified """
        options = {'format':'csv', 'header':header}
        df = spark.read.load(self.to_url(source_path), **options)
        return df      

    def load_json(self, source_path, multiline=False):
        """ Loads a json file as a dataframe based on the path specified """
        options = {'format':'json', 'multiline':multiline}
        df = spark.read.load(self.to_url(source_path), **options)
        return df    

    def pseudonymize(self, df, metadata): #: list[list[str]]):
        """ Performs pseudonymization of the given dataframe based on the provided metadata (in the OEA format).
            For example, if the given df is for an entity called person, 
            2 dataframes will be returned, one called person that has hashed ids and masked fields, 
            and one called person_lookup that contains the original person_id, person_id_pseudo,
            and the non-masked values for columns marked to be masked.           
            The lookup table should be written to a "sensitive" folder in the data lake.
            eg, df_pseudo, df_lookup = oea.pseudonymize(df, metadata)
            [More info on this approach here: https://learn.microsoft.com/en-us/azure/databricks/security/privacy/gdpr-delta#pseudonymize-data]
        """
        salt = self._get_salt()
        df_pseudo = df
        df_lookup = df
        for row in metadata:
            col_name = row[0]
            dtype = row[1]
            op = row[2]
            if op == "hash-no-lookup" or op == "hnl":
                # This means that the lookup can be performed against a different table so no lookup is needed.
                df_pseudo = df_pseudo.withColumn(col_name, F.sha2(F.concat(F.col(col_name), F.lit(salt)), 256)).withColumnRenamed(col_name, col_name + "_pseudonym")
                df_lookup = df_lookup.drop(col_name)           
            elif op == "hash" or op == 'h':
                df_pseudo = df_pseudo.withColumn(col_name, F.sha2(F.concat(F.col(col_name), F.lit(salt)), 256)).withColumnRenamed(col_name, col_name + "_pseudonym")
                df_lookup = df_lookup.withColumn(col_name + "_pseudonym", F.sha2(F.concat(F.col(col_name), F.lit(salt)), 256))
            elif op == "mask" or op == 'm':
                df_pseudo = df_pseudo.withColumn(col_name, F.lit('*'))
            elif op == "partition-by":
                pass # make no changes for this column so that it will be in both dataframes and can be used for partitioning
            elif op == "no-op" or op == 'x':
                df_lookup = df_lookup.drop(col_name)
        return (df_pseudo, df_lookup)

    def add_to_lake_db(self, source_entity_path):
        """ Adds the given entity as a table (if the table doesn't already exist) to the proper lake db based on the path.
            This method will also create the lake db if it doesn't already exist.
            eg: add_to_lake_db('stage2/Ingested/contoso_sis/v0.1/students')

            Note that a spark db that points to source data in the delta format can't be queried via SQL serverless pool. More info here: https://docs.microsoft.com/en-us/azure/synapse-analytics/sql/resources-self-help-sql-on-demand#delta-lake
        """
        source_dict = self.parse_path(source_entity_path)
        db_name = source_dict['ldb_name']
        spark.sql(f'CREATE DATABASE IF NOT EXISTS {db_name}')
        spark.sql(f"create table if not exists {db_name}.{source_dict['entity']} using DELTA location '{self.to_url(source_dict['entity_path'])}'")

    def create_lake_db(self, dataset_full_path):
        """ Creates the lake db for the given dataset, creating an external table for each entity.
            eg: create_lake_db('stage2/Ingested/contoso/v0.1') # creates a lake db named ldb_sam_s2i_contoso_v0p1
        """
        folders = self.get_folders(self.to_url(dataset_full_path))
        for entity_name in folders:
            number_of_new_inbound_rows += self.add_to_lake_db(f'{dataset_full_path}/{entity_name}')


    def drop_lake_db(self, db_name):
        spark.sql(f'DROP DATABASE IF EXISTS {db_name} CASCADE')
        result = "Database dropped: " + db_name
        logger.info(result)
        return result

    def create_sql_db(self, source_path):
        """ Prints out the sql script needed for creating a sql serverless db and set of views. """
        source_dict = self.parse_path(source_path)
        db_name = source_dict['sdb_name']
        cmd = '-- Create a new sql script then execute the following in it:\n'
        cmd += f"IF NOT EXISTS (SELECT * FROM sys.databases WHERE name = '{db_name}')\nBEGIN\n  CREATE DATABASE {db_name};\nEND;\nGO\n"
        cmd += f"USE {db_name};\nGO\n\n"
        cmd += self.create_sql_views(source_dict['entity_parent_path'])
        print(cmd)

    def create_sql_views(self, source_path):
        cmd = ''      
        dirs = self.get_folders(source_path)
        for table_name in dirs:
            cmd += f"CREATE OR ALTER VIEW {table_name} AS\n  SELECT * FROM OPENROWSET(BULK '{self.to_url(source_path)}/{table_name}', FORMAT='delta') AS [r];\nGO\n"
        return cmd 

    def drop_sql_db(self, db_name):
        cmd = '-- Create a new sql script then execute the following in it. Alternatively, you can click on the menu next to the SQL db and select "Delete"\n'
        cmd += '-- [Note that this does not affect the data in the data lake - this will only delete the sql db that points to that data.]\n\n'
        cmd += f'DROP DATABASE {db_name}'
        print(cmd)       

    def create_metadata_from_lake_db(self, db_name):
        buffer = 'Entity Name,Attribute Name,Attribute Data Type,Pseudonymization'
        tables = spark.catalog.listTables(db_name)
        for table in tables:     
            buffer += f'{table.name},,,\n'
            df = spark.table(f'{db_name}.{table.name}')
            for name, dtype in df.dtypes:
                if 'struct' in dtype: dtype = 'string'
                buffer += f',{name},{dtype},no-op\n'
        return buffer

    def to_simple_schema_spec(self, *dataset_full_paths):
        """ Creates a simple-schema-spec for the delta lake at the given path(s).
            This is used when creating a schema component for the Github repo based on an existing delta table.
            see https://github.com/microsoft/OpenEduAnalytics/tree/main/schemas
            
            eg: to_simple_schema_spec('stage2/Refined/M365/v1.14/general', 'stage2/Refined/M365/v1.14/sensitive')
        """
        csv_str = 'Entity Name,Attribute Name,Attribute Data Type,nullable\n'
        path_dict = self.parse_path(dataset_full_paths[0])
        destination_path = f"stage1/{path_dict['source_system']}.sss.csv"

        for dataset_full_path in dataset_full_paths:
            folders = self.get_folders(self.to_url(dataset_full_path))
            for entity_name in folders:
                csv_str += f'{entity_name},,,\n'
                df = self.load(f'{dataset_full_path}/{entity_name}')
                for attribute in df.schema:
                    values = attribute.jsonValue()
                    csv_str += f",{values['name']},{values['type']},{values['nullable']}\n"

        oea.write(csv_str, destination_path)
        logger.info(f'Simple schema spec written to {destination_path}')   

    def export_data(self, *dataset_full_paths):
        """ Exports data from delta tables in the specified paths. Writes exported data to a new folder called 'extracted_data'.
            The primary purpose for this method is to facilitate the extraction of test data to be used in an OEA schema.
            see https://github.com/microsoft/OpenEduAnalytics/tree/main/schemas

            eg: export_data('stage2/Refined/M365/v1.14/general', 'stage2/Refined/M365/v1.14/sensitive')
        """
        destination_path = '/'.join(dataset_full_paths[0].split('/')[0:-1]) + '/extracted_data'
        self.rm_if_exists(destination_path)
        tmp_path = f'{destination_path}/tmp'
        for dataset_full_path in dataset_full_paths:
            logger.info(f'Gathering data from {dataset_full_path} to {destination_path}')
            folders = self.get_folders(self.to_url(dataset_full_path))
            for folder in folders:
                df = self.load(f'{dataset_full_path}/{folder}')
                df.coalesce(1).write.mode('overwrite').parquet(self.to_url(tmp_path)) # write as a single parquet file
                filename = mssparkutils.fs.ls(self.to_url(tmp_path))[1].name
                mssparkutils.fs.mv(f'{self.to_url(tmp_path)}/{filename}', f'{self.to_url(destination_path)}/{folder}.snappy.parquet')
        self.rm_if_exists(tmp_path)

    def _create_empty_delta_table(self, folder_path, simple_schema_spec):
        """ Creates an empty delta table in the specified path with the schema specified by simple_schema_spec.
        """
        # todo: if simple_schema_spec has a 'description' column, use 'ALTER COLUMN' to add COMMENT at the attribute level
        folder_url = self.to_url(folder_path)
        logger.info(f'Creating table at: {folder_url}')
        spark_schema = self.to_spark_schema(simple_schema_spec)
        if not DeltaTable.isDeltaTable(spark, folder_url):
            emptyDF = spark.createDataFrame(spark.sparkContext.emptyRDD(), spark_schema)
            emptyDF.write.format('delta').mode('overwrite').save(folder_url)

    def _insert_test_data_to_delta_table(self, test_data_url, destination_path, simple_schema_spec):
        """ Retrieves test data in parquet format from Github and writes it to the delta tables in the specified destination_path.
        """
        logger.info(f'Inserting test data from: {test_data_url}\ninto {destination_path}')
        pdf = pd.read_parquet(test_data_url)
        df = spark.createDataFrame(pdf)
        spark_schema = self.to_spark_schema(simple_schema_spec)
        df = self.modify_schema(df, spark_schema)
        df.write.mode('overwrite').format('delta').save(self.to_url(destination_path))

    def install_schema(self, sss_url, destination_path, insert_test_data = False):
        """
        Installs a schema from Github.
        Uses the given url to retrieve the simple schema spec and create empty delta lake tables as defined by the given schema.
        If the insert_test_data flag is True, this method assumes that there is also a test_data folder in given repo url with test data in parquet format.
        It retrieves the test data from Github and writes it to the newly created delta lake tables.
        And then a lake db is created to make it easy to work with inside Synapse.

        We assume that the schema is being created in stage2 or stage3, and that tables that end with '_lookup' need to be written to the 'sensitive' folder
        and all other tables need to be written to 'general'.

        eg: install_schema('https://raw.githubusercontent.com/microsoft/OpenEduAnalytics/gene/schema_as_component/schemas/schema_catalog/Microsoft_Education_Insights/Microsoft_Education_Insights.sss.csv', 'stage2/Refined/contoso6/v0.1', True)
        """
        logger.info(f'Installing schema as defined by: {sss_url}\nInstalling into {destination_path}\nFlag to insert test data is set to: {insert_test_data}')
        test_data_url = '/'.join(sss_url.split('/')[:-1]) + '/test_data' # remove the last part of the path and add 'test_data'
        sss = self.get_metadata_from_url(sss_url)
        for tablename in sss.keys():
            if tablename.endswith('_lookup'):
                destination_table_path = f'{destination_path}/sensitive/{tablename}'
            else:
                destination_table_path = f'{destination_path}/general/{tablename}'

            self._create_empty_delta_table(destination_table_path, sss[tablename])
            if insert_test_data:
                self._insert_test_data_to_delta_table(f'{test_data_url}/{tablename}.snappy.parquet', destination_table_path, sss[tablename])

            # now add it to the lake db in synapse (note that the lake db will be created by self if it doesn't alread exist)
            self.add_to_lake_db(destination_table_path)
        logger.info('Schema installed - lake db created.')

class DataLakeWriter:
    def __init__(self, root_destination):
        self.root_destination = root_destination

    def write(self, path_and_filename, data_str, format='csv'):
        mssparkutils.fs.append(f"{self.root_destination}/{path_and_filename}", data_str, True) # Set the last parameter as True to create the file if it does not exist

class OpenAPIUtil:
    """
    A Utility class to help processing transformations using Open API (Swagger).

    Parameters:
        1) swagger_url: URL to the OpenAPI Swaggern endpoint

    Methods:
        1) create_spark_schemas(): returns a dictionary of Spark schemas of all endpoints with entity name as the Key.
        2) create_metadata(): returns list of dictionaries containing metadata of each field in every endpoint.
        3) write_oea_metadata(destination_path): Writes out the OEA Metadata CSV file at the destination directory.
    """

    def __init__(self, swagger_url):
        self.swagger_json = json.loads(requests.get(swagger_url).text)
        self.metadata_headers = ['table_name', 'column_name', 'type', 'format', 'maxLength', 'required', 'items', '$ref', 'pseudonymization']
        self.definitions = {}
        self.metadata = {}
        self.tables = []
        self.schemas = {}
        self.dependency_dict = {}
        self.dependency_order = []
        self.visited = {}

    def get_reference(self, row):
        if(row['type'] == 'array'):
            reference = row['items']['$ref']
        elif(row['$ref'] != None):
           reference = row['$ref']
        else:
            return None
        return reference.split('/')[-1].split('_')[-1]

    def pluralize(self, noun):
        if noun == 'person' : return 'people'
        if noun == 'survey' : return 'surveys'
        if re.search('[sxz]$', noun):
            return re.sub('$', 'es', noun)
        if re.search('y$', noun):
            return re.sub('y$', 'ies', noun)
        return noun + 's'

    def get_data_type(self, dtype, format):
        if(dtype == 'string'):
            if(format == 'date'):
                return DateType()
            if(format == 'date-time'):
                return TimestampType()
            return StringType()
        if(dtype == 'integer'):
            return IntegerType()
        if(dtype == 'number'):
            return DecimalType()
        if(dtype == 'boolean'):
            return BooleanType()

    def create_definitions(self):
        for entity in self.swagger_json['definitions']:
            properties = self.swagger_json['definitions'][entity]['properties']
            table_name = entity.split('_')[-1]
            table_schema = {}

            for prop in properties:
                if 'description' in properties[prop].keys():
                    properties[prop].pop('description')
                field_info = properties[prop]
                if 'required' in self.swagger_json['definitions'][entity].keys():
                    field_info['required'] = True if prop in self.swagger_json['definitions'][entity]['required'] else False
                else:
                    field_info['required'] = False
                field_info['table_name'] = entity.split('_')[-1]
                field_info['column_name'] = prop
                if 'x-Ed-Fi-pseudonymization' in field_info:
                    field_info['pseudonymization'] = field_info['x-Ed-Fi-pseudonymization']
                    field_info.pop('x-Ed-Fi-pseudonymization')
                for header in [x for x in self.metadata_headers if x not in field_info] : field_info[header] = None
                table_schema[prop] = field_info

            self.definitions[table_name] = table_schema
        self.tables = [x for x in self.definitions.keys()]

    def create_metadata(self):
        if(len(self.schemas) == 0):
            self.create_spark_schemas()
        for table_name in self.dependency_order:
            table_metadata = []
            for col_name in self.definitions[table_name]:
                col_schema = self.definitions[table_name][col_name]
                key = self.pluralize(table_name)

                if 'x-Ed-Fi-fields-to-pluck' in col_schema and col_schema['x-Ed-Fi-fields-to-pluck'] != ["*"]:
                    referenced_table = self.get_reference(col_schema)
                    table_metadata += [x for x in self.metadata[self.pluralize(referenced_table)] if x[0] in col_schema['x-Ed-Fi-fields-to-pluck']]

                elif 'x-Ed-Fi-explode' in col_schema and col_schema['x-Ed-Fi-explode']:
                    referenced_table = self.get_reference(col_schema)
                    table_metadata += self.metadata[self.pluralize(referenced_table)]

                else:
                    op = self.definitions[table_name][col_name]['pseudonymization']
                    if op == None: op = 'no-op'
                    table_metadata.append([col_name, self.schemas[key][col_name].dataType.typeName(), op])
            self.metadata[self.pluralize(table_name)] = table_metadata
        return self.metadata

    def write_oea_metadata(self, destination_path):
        if(self.metadata == []):
            self.create_metadata()
        oea_metadata = []
        for table_name in self.metadata:
            oea_metadata.append([table_name, None, None, None])
            for col_metadata in self.metadata[table_name]:
                oea_metadata.append([None] + col_metadata)
        metadata_df = spark.createDataFrame(oea_metadata, ['Entity Name','Attribute Name','Attribute Data Type','Pseudonymization'])
        metadata_df.coalesce(1).write.format('csv').save(destination_path)

    def create_dependency_dict(self):
        for table_name in self.definitions:
            for column_name in self.definitions[table_name]:
                column_info = self.definitions[table_name][column_name]
                referenced_table = self.get_reference(column_info)
                if(referenced_table is None):
                    continue
                if table_name not in self.dependency_dict:
                    self.dependency_dict[table_name] = [referenced_table]
                elif referenced_table not in self.dependency_dict[table_name]:
                    self.dependency_dict[table_name].append(referenced_table)

    def dfs(self, table_name):
        self.visited[table_name] = True
        if table_name not in self.dependency_dict:
            self.dependency_order.append(table_name)
            return

        for dependent_table in self.dependency_dict[table_name]:
            if(self.visited[dependent_table] is False):
                self.dfs(dependent_table)
            if(self.visited[dependent_table] is False):
                self.dependency_order.append(dependent_table)

        self.dependency_order.append(table_name)

    def create_dependency_order(self):
        for table_name in self.tables:
            self.visited[table_name] = False
        for table_name in self.tables:
            if(self.visited[table_name] is False):
                self.dfs(table_name)

    def create_spark_schemas_from_definitions(self):
        for entity in self.dependency_order:
            table_schema = self.definitions[entity]
            spark_schema = []
            if(entity == 'localEducationAgencyReference'):
                print(entity)
            for col_name in table_schema:
                col_metadata = {}
                if('pseudonymization' in table_schema[col_name]): col_metadata['pseudonymization'] = table_schema[col_name]['pseudonymization']
                if('x-Ed-Fi-isIdentity' in table_schema[col_name]): col_metadata['x-Ed-Fi-isIdentity'] = table_schema[col_name]['x-Ed-Fi-isIdentity']
                
                col_metadata['required'] = table_schema[col_name]['required']
                referenced_table = self.get_reference(table_schema[col_name])
                if table_schema[col_name]['type'] == 'array':
                    datatype = ArrayType(self.schemas[self.pluralize(referenced_table)])
                    if('x-Ed-Fi-explode' in table_schema[col_name]):
                        col_metadata['x-Ed-Fi-explode'] = table_schema[col_name]['x-Ed-Fi-explode']
                elif table_schema[col_name]['$ref'] != None:
                    datatype = self.schemas[self.pluralize(referenced_table)]
                    if('x-Ed-Fi-fields-to-pluck' in table_schema[col_name]):
                        col_metadata['x-Ed-Fi-fields-to-pluck'] = table_schema[col_name]['x-Ed-Fi-fields-to-pluck']
                else:
                    datatype = self.get_data_type(table_schema[col_name]['type'], table_schema[col_name]['format'])
                col_spark_schema = StructField(col_name, datatype, not(table_schema[col_name]['required']))
                col_spark_schema.metadata = col_metadata
                spark_schema.append(col_spark_schema)
            self.schemas[self.pluralize(entity)] = StructType(spark_schema)

    def create_spark_schemas(self):
        if(len(self.schemas) == 0):
            self.create_definitions()
            self.create_dependency_dict()
            self.create_dependency_order()
            self.create_spark_schemas_from_definitions()
        return self.schemas

oea = OEA()